<a href="https://colab.research.google.com/github/Yundo37/automation-portfolio-Yundo/blob/main/Test_Automation_Yundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== 기본 설정 =====
print("Selenium 테스트 자동화 포트폴리오")
print("QA Engineer: 윤도현")

# 환경 설정
import subprocess
import sys
import os

subprocess.run(['apt-get', 'update', '-qq'], capture_output=True)
subprocess.run(['apt-get', 'install', '-y', '-qq', 'chromium-browser'], capture_output=True)
subprocess.check_call([sys.executable, "-m", "pip", "install", "selenium", "pytest", "pytest-html", "webdriver-manager", "-q"])

# 폴더 생성
for folder in ['tests', 'reports']:
    os.makedirs(folder, exist_ok=True)

print("✓ 환경 설정 완료")

In [ ]:
# ===== E2E 테스트 코드 =====
print("\n📋 E2E 테스트 코드 (test_e2e.py)")
print("="*50)

test_e2e_code = '''import pytest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

@pytest.fixture(scope="class")
def setup(request):
    print("\\n브라우저 실행 중...")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://rahulshettyacademy.com/angularpractice/")
    driver.implicitly_wait(5)

    request.cls.driver = driver
    yield
    driver.quit()

@pytest.mark.usefixtures("setup")
class TestE2E:
    def test_e2e_purchase(self):
        print("\\n=== E2E 테스트 시작: 전체 주문 프로세스 ===")

        print("1. Shop 메뉴로 이동...")
        self.driver.find_element(By.CSS_SELECTOR, "a[href*='shop']").click()

        print("2. 상품 목록 조회...")
        products = self.driver.find_elements(By.XPATH, "//div[@class='card h-100']")
        print(f"   총 {len(products)}개 상품 발견")

        print("3. Blackberry 상품 검색...")
        for product in products:
            name = product.find_element(By.XPATH, "div/h4/a").text
            print(f"   - 상품: {name}")
            if name == "Blackberry":
                print("   ✓ Blackberry 발견! 장바구니 추가")
                product.find_element(By.XPATH, "div/button").click()
                break

        print("4. 장바구니로 이동...")
        self.driver.find_element(By.CSS_SELECTOR, "a[class*=btn-primary]").click()

        print("5. 체크아웃...")
        self.driver.find_element(By.XPATH, "//button[@class='btn btn-success']").click()

        print("6. 배송 정보 입력...")
        self.driver.find_element(By.ID, "country").send_keys("ind")

        wait = WebDriverWait(self.driver, 10)
        india = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "India")))
        india.click()
        print("   - 국가: India 선택")

        self.driver.find_element(By.XPATH, "//div[@class='checkbox checkbox-primary']").click()
        print("   - 약관 동의")

        print("7. 주문 제출...")
        self.driver.find_element(By.CSS_SELECTOR, "[type='submit']").click()

        success = self.driver.find_element(By.CLASS_NAME, "alert-success").text
        print(f"8. 주문 완료: {success}")

        assert "Success! Thank you!" in success
        print("=== E2E 테스트 성공! ===\\n")
'''

print(test_e2e_code)

with open('tests/test_e2e.py', 'w') as f:
    f.write(test_e2e_code)

In [ ]:
# ===== 폼 테스트 코드 =====
print("\n📋 폼 테스트 코드 (test_homepage.py)")
print("="*50)

test_homepage_code = '''import pytest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

# 테스트 데이터
test_data = [
    {"username": "윤도현", "email": "test1@example.com", "gender": "Male"},
    {"username": "테스트", "email": "test2@example.com", "gender": "Female"},
    {"username": "실패테스트", "email": "fail@test.com", "gender": "InvalidGender"}
]

@pytest.fixture(scope="class")
def setup(request):
    print("\\n브라우저 실행 중...")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://rahulshettyacademy.com/angularpractice/")
    driver.implicitly_wait(5)

    request.cls.driver = driver
    yield
    driver.quit()

@pytest.mark.usefixtures("setup")
class TestHomePage:
    @pytest.mark.parametrize("data", test_data)
    def test_form_submission(self, data):
        print(f"\\n=== 폼 테스트: {data['username']} ===")

        print("1. 사용자 정보 입력...")
        self.driver.find_element(By.CSS_SELECTOR, "input[name='name']").send_keys(data["username"])
        print(f"   - 이름: {data['username']}")

        self.driver.find_element(By.NAME, "email").send_keys(data["email"])
        print(f"   - 이메일: {data['email']}")

        self.driver.find_element(By.ID, "exampleInputPassword1").send_keys("123456")
        print("   - 패스워드: ******")

        print("2. 옵션 선택...")
        self.driver.find_element(By.ID, "exampleCheck1").click()
        self.driver.find_element(By.CSS_SELECTOR, "#inlineRadio1").click()
        print("   - 체크박스 및 라디오 버튼 선택")

        print(f"3. 성별 선택: {data['gender']}")
        if data["gender"] == "InvalidGender":
            print("   ⚠ 의도적 실패 케이스 (스크린샷 캡처)")

        dropdown = Select(self.driver.find_element(By.ID, "exampleFormControlSelect1"))
        dropdown.select_by_visible_text(data["gender"])

        print("4. 폼 제출...")
        self.driver.find_element(By.XPATH, "//input[@type='submit']").click()

        message = self.driver.find_element(By.CLASS_NAME, "alert-success").text
        print(f"5. 결과: {message}")

        assert "Success" in message
        self.driver.refresh()
        print("=== 테스트 완료 ===\\n")
'''

print(test_homepage_code)

with open('tests/test_homepage.py', 'w') as f:
    f.write(test_homepage_code)


In [ ]:
# ===== 테스트 실행 =====
print("\n🚀 테스트 실행!")
print("="*50)

# E2E 테스트 실행
print("📌 E2E 테스트 실행:")
result1 = subprocess.run([sys.executable, "-m", "pytest", "tests/test_e2e.py", "-v", "-s"])

print("\n📌 폼 테스트 실행:")
result2 = subprocess.run([sys.executable, "-m", "pytest", "tests/test_homepage.py", "-v", "-s"])

print(f"\n✓ 테스트 완료!")
print("연락처: porore37@naver.com")